In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Delta-Fix")
    .master("spark://spark-master:7077")
    .getOrCreate()
)


In [2]:
spark

In [3]:
conf = spark._jsc.hadoopConfiguration()
print(conf.get("fs.s3a.path.style.access"))
print("S3 Endpoint:", conf.get("fs.s3a.endpoint"))
print("Access Key:", conf.get("fs.s3a.access.key"))
print("Secret Key:", conf.get("fs.s3a.secret.key"))

true
S3 Endpoint: http://minio:9000
Access Key: minio_access_key
Secret Key: minio_secret_key


In [4]:
spark.sql("SHOW DATABASES").show(truncate=False)


Py4JJavaError: An error occurred while calling o43.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catalog.CatalogManage

In [ ]:
df = spark.read.option("header", True).csv(src_csv)

In [16]:


df2 = df.dropna(subset=["CustomerID"]).dropDuplicates(["CustomerID"])
df2.write.mode("overwrite").parquet(dst_parquet)
spark.stop

NameError: name 'df' is not defined

In [17]:
spark.conf.get("spark.sql.catalog.spark_catalog")


'org.apache.spark.sql.delta.catalog.DeltaCatalog'